In [41]:
#coding=utf-8
import tushare as ts
import pandas as pd
import talib
import time
import datetime
from opendatatools import swindex
from os import walk

ts.set_token('bc406c9fab768c6c699ef450685daed298c5abf2dbdba8da4e810cab')
pro = ts.pro_api()


In [21]:
import baostock as bs
import pandas as pd

# 登陆系统
lg = bs.login()
# 显示登陆返回信息
print('login respond error_code:'+lg.error_code)
print('login respond  error_msg:'+lg.error_msg)

# 获取指数(综合指数、规模指数、一级行业指数、二级行业指数、策略指数、成长指数、价值指数、主题指数)K线数据
# 综合指数，例如：sh.000001 上证指数，sz.399106 深证综指 等；
# 规模指数，例如：sh.000016 上证50，sh.000300 沪深300，sh.000905 中证500，sz.399001 深证成指等；
# 一级行业指数，例如：sh.000037 上证医药，sz.399433 国证交运 等；
# 二级行业指数，例如：sh.000952 300地产，sz.399951 300银行 等；
# 策略指数，例如：sh.000050 50等权，sh.000982 500等权 等；
# 成长指数，例如：sz.399376 小盘成长 等；
# 价值指数，例如：sh.000029 180价值 等；
# 主题指数，例如：sh.000015 红利指数，sh.000063 上证周期 等；

# 详细指标参数，参见“历史行情指标参数”章节
rs = bs.query_history_k_data_plus("000906.sh",
    "date,code,open,high,low,close,preclose,volume,amount,pctChg",
    start_date='2014-01-01', end_date='2020-10-23', frequency="d")
print('query_history_k_data_plus respond error_code:'+rs.error_code)
print('query_history_k_data_plus respond  error_msg:'+rs.error_msg)

# 打印结果集
data_list = []
while (rs.error_code == '0') & rs.next():
    # 获取一条记录，将记录合并在一起
    data_list.append(rs.get_row_data())
result = pd.DataFrame(data_list, columns=rs.fields)
result.to_csv("C:/quant/index/history_Index_k_data.csv", index=False)# 登出系统
bs.logout()

login success!
login respond error_code:0
login respond  error_msg:success
query_history_k_data_plus respond error_code:0
query_history_k_data_plus respond  error_msg:success
logout success!


In [22]:
#因为baostock出数据慢，手动输入当日数据后也能算MA

result=pd.read_csv("C:/quant/index/history_Index_k_data.csv")# 登出系统

ma5=talib.SMA(result.close,timeperiod=5)
ma10=talib.SMA(result.close,timeperiod=10)
ma20=talib.SMA(result.close,timeperiod=20)
ma30=talib.SMA(result.close,timeperiod=30)
ma60=talib.SMA(result.close,timeperiod=60)
ma120=talib.SMA(result.close,timeperiod=120)

ema5=talib.SMA(result.close,timeperiod=5)
ema10=talib.SMA(result.close,timeperiod=10)
ema20=talib.SMA(result.close,timeperiod=20)
ema30=talib.SMA(result.close,timeperiod=30)
ema60=talib.SMA(result.close,timeperiod=60)
ema120=talib.SMA(result.close,timeperiod=120)


result['MA5']=ma5
result['MA10']=ma10
result['MA20']=ma20
result['MA30']=ma30
result['MA60']=ma60
result['MA120']=ma120
result['EMA5']=ema5
result['EMA10']=ema10
result['EMA20']=ema20
result['EMA30']=ema30
result['EMA60']=ema60
result['EMA120']=ema120


# 结果集输出到csv文件
result.to_csv("C:/quant/index/history_Index_k_data.csv", index=False)
print(result)

# 登出系统
bs.logout()

            date       code       open       high        low      close  \
0     2014-01-02  sh.000906  2582.9100  2586.9400  2570.9500  2586.0200   
1     2014-01-03  sh.000906  2576.6100  2579.2900  2544.5900  2554.8000   
2     2014-01-06  sh.000906  2549.4700  2549.4700  2487.0700  2493.4000   
3     2014-01-07  sh.000906  2478.0200  2504.6600  2473.2700  2496.8600   
4     2014-01-08  sh.000906  2499.3500  2523.1300  2488.0000  2500.8800   
...          ...        ...        ...        ...        ...        ...   
1653  2020-10-19  sh.000906  5089.7820  5112.9990  5005.9923  5014.7910   
1654  2020-10-20  sh.000906  5009.8481  5057.4922  4996.0496  5057.4922   
1655  2020-10-21  sh.000906  5064.6191  5064.6191  5011.8987  5043.1498   
1656  2020-10-22  sh.000906  5029.8992  5039.0519  4971.4497  5025.0332   
1657  2020-10-23  sh.000906  5022.0431  5049.3727  4954.5464  4957.6898   

       preclose       volume        amount    pctChg  ...         MA20  \
0     2589.3500   7599455

In [23]:

pool = pro.stock_basic()
pool.head(5)
writer = pd.ExcelWriter('C:/quant/list/list.xlsx') 
pool.to_excel(writer,'Sheet1')
writer.save()
writer.close()


In [37]:
#从tushare模块获取申银三级行业列表
SWf= pro.index_basic(market='SW')

S1=SWf.loc[SWf['category']=='三级行业指数']
#lc.loc[lc["grade"] == "B"].head()

#从申银端口通过刚才过滤的三级行业列表获取数据

for i in S1.ts_code:
    time.sleep(0.01)
    X=str(i)
    df2 = pro.index_daily(ts_code=X, start_date='20200101', end_date='20201029')
    #df2, msg = swindex.get_index_daily(X,start_date=20200101,end_date=20201029)
    df2.to_csv(r'C:/quant/indexdaydate/2020/'+X+'.csv',index=False,encoding='utf-8-sig') 
  


,ts_code,name,market,publisher,category,base_date,base_point,list_date
238,850111.SI,种子生产(申万),SW,申万研究,三级行业指数,19991230,1000.0,None
239,850112.SI,粮食种植(申万),SW,申万研究,三级行业指数,19991230,1000.0,None
240,850113.SI,其他种植业(申万),SW,申万研究,三级行业指数,19991230,1000.0,None
241,850121.SI,海洋捕捞(申万),SW,申万研究,三级行业指数,19991230,1000.0,None
242,850122.SI,水产养殖(申万),SW,申万研究,三级行业指数,19991230,1000.0,None
...,...,...,...,...,...,...,...,...
502,857344.SI,线缆部件及其他(申万),SW,申万研究,三级行业指数,19991230,1000.0,20140221
503,857411.SI,航天装备Ⅲ(申万),SW,申万研究,三级行业指数,19991230,1000.0,20140221
504,857421.SI,航空装备Ⅲ(申万),SW,申万研究,三级行业指数,19991230,1000.0,20140221
505,857431.SI,地面兵装Ⅲ(申万),SW,申万研究,三级行业指数,19991230,1000.0,20140221


In [40]:
#从申银端口通过刚才过滤的一级行业列表获取数据
allindex = pd.DataFrame() 

for i in S1.ts_code:
    time.sleep(0.01)
    X=str(i)
    df2 = pro.index_daily(ts_code=X, start_date='20200101', end_date='20201029')
    #df2, msg = swindex.get_index_daily(X,start_date=20200101,end_date=20201029)
    df2.to_csv(r'C:/quant/indexdaydate/2020/'+X+'.csv',index=False,encoding='utf-8-sig') 

    #allindex = allindex.append(df2)
#allindex = allindex.sort_values(by=['index_code','date'],axis=0,ascending=[True,True])
#allindex
   


In [ ]:
for root,dirs,files in walk(r'C:/quant/indexdaydate/2020/',topdown=False):
    num = len(files)
    allindexdata = pd.DataFrame() 
for i in range(num):
    newIdata = pd.read_csv(r'C:/quant/indexdaydate/2020/%s'%files[i])
    
    allindexdata = allindexdata.append(newIdata)
allindexdata = allindexdata.sort_values(by=['ts_code','trade_date_x'],axis=0,ascending=[True,True])
allindexdata.reset_index(drop=True, inplace=True)


In [30]:
#申银源数据格式不对，转换格式
allindex['open']=allindex['open'].astype("float")
allindex['high']=allindex['high'].astype("float")
allindex['low']=allindex['low'].astype("float")
allindex['close']=allindex['close'].astype("float")
allindex['vol']=allindex['vol'].astype("float")
allindex['amount']=allindex['amount'].astype("float")
allindex['change_pct']=allindex['change_pct'].astype("float")
allindex.isnull().sum()


index_code    0
index_name    0
date          0
open          0
high          0
low           0
close         0
vol           0
amount        0
change_pct    0
dtype: int64

In [31]:
#计算申银行业指数MA，MACD
from scipy.stats import rankdata
swsf=allindex[allindex['index_code'].notna()]

close=swsf.close.values
swsf.date=swsf.date.values

diff, dea, my_macd = talib.MACD(close,
                                             fastperiod=12,
                                             slowperiod=26,
#获取均线的数据，通过timeperiod参数来分别获取 5,10,20 日均线的数据。
                                             signalperiod=9)
SMA5=talib.SMA(close,timeperiod=5)
SMA10=talib.SMA(close,timeperiod=10)
SMA20=talib.SMA(close,timeperiod=20)
SMA60=talib.SMA(close,timeperiod=60)
SMA120=talib.SMA(close,timeperiod=120)
SMA350=talib.SMA(close,timeperiod=350)

EMA5=talib.EMA(close,timeperiod=5)
EMA10=talib.EMA(close,timeperiod=10)
EMA20=talib.EMA(close,timeperiod=20)
EMA60=talib.EMA(close,timeperiod=60)
EMA120=talib.EMA(close,timeperiod=120)

swsf['diff']=diff
swsf['dea']=dea
swsf['MACD']=my_macd
swsf['SMA5']=SMA5
swsf['SMA10']=SMA10
swsf['SMA20']=SMA20
swsf['SMA60']=SMA60
swsf['SMA120']=SMA120
swsf['SMA350']=SMA350

swsf['EMA5']=EMA5
swsf['EMA10']=EMA10
swsf['EMA60']=EMA60
swsf['EMA120']=EMA120

swsf['EMA20']=EMA20
swsf['SMACD-K']=(diff-dea)/dea
swsf['C/S']=(close/SMA20-1)*100
swsf['M/L']=(SMA60/SMA120-1)*100
swsf['S/M']=(SMA20/SMA60-1)*100
swsf['BreakSignal-20D']=close/EMA20-1
swsf.reset_index(drop=True, inplace=True)

swsf['dailyamount']=swsf.groupby('date')['amount'].sum()
swsf['indexamount']=swsf.amount/swsf.dailyamount


swsf.loc[swsf['index_code'] == swsf['index_code'].shift(1),'Chg1D']= (swsf.close/swsf.close.shift(1)-1)*100
swsf.loc[swsf['index_code'] == swsf['index_code'].shift(4),'Chg5D']= (swsf.close/swsf.close.shift(4)-1)*100
swsf.loc[swsf['index_code'] == swsf['index_code'].shift(19),'Chg20D']= (swsf.close/swsf.close.shift(19)-1)*100

swsf.fillna(0,inplace=True) #去除null
swsf.loc[(swsf['close']>swsf['EMA20']) & (swsf['close']>swsf['SMA20']) & (swsf['SMA20']>swsf['SMA20'].shift(19)),'K1']=1 #破线拐头
swsf.loc[(swsf['EMA20']>swsf['EMA60']) & (swsf['close']>swsf['EMA20'].shift(19)),'K2']=1 #双均线交叉
swsf.loc[(swsf['EMA20']>swsf['SMA60']) & (swsf['close']>swsf['EMA20'].shift(19)),'K3']=1 
swsf.loc[(swsf['SMA20']>swsf['EMA60']) & (swsf['close']>swsf['SMA20'].shift(19)),'K4']=1 
swsf.loc[(swsf['SMA20']>swsf['SMA60']) & (swsf['close']>swsf['SMA20'].shift(19)),'K5']=1 

swsf.loc[(swsf['close']<swsf['EMA20']) & (swsf['close']<swsf['SMA20']) & (swsf['SMA20']<swsf['SMA20'].shift(19)),'K1']=-1 #空头方向破线拐头
swsf.loc[(swsf['EMA20']<swsf['EMA60']) & (swsf['close']<swsf['EMA20'].shift(19)),'K2']=-1 #双均线交叉
swsf.loc[(swsf['EMA20']<swsf['SMA60']) & (swsf['close']<swsf['EMA20'].shift(19)),'K3']=-1 
swsf.loc[(swsf['SMA20']<swsf['EMA60']) & (swsf['close']<swsf['SMA20'].shift(19)),'K4']=-1 
swsf.loc[(swsf['SMA20']<swsf['SMA60']) & (swsf['close']<swsf['SMA20'].shift(19)),'K5']=-1 
swsf.fillna(0,inplace=True) #去除null
swsf['Indextrendscore-L']=(swsf.K1+swsf.K2+swsf.K3+swsf.K4+swsf.K5)

swsf.head(10)
#swsf.to_csv('C:/quant/index/SWoutput.csv',index=False,encoding='utf-8-sig') 

,index_code,index_name,date,open,high,low,close,vol,amount,change_pct,...,indexamount,Chg1D,Chg5D,Chg20D,K1,K2,K3,K4,K5,Indextrendscore-L
0,801010,农林牧渔,2020-01-02,3347.27,3408.34,3345.43,3394.96,16.96,210.61,2.63,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,801010,农林牧渔,2020-01-03,3420.59,3436.81,3392.78,3425.51,18.30,195.75,0.90,...,0.0,0.899863,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,801010,农林牧渔,2020-01-06,3426.80,3509.56,3426.74,3494.01,20.19,205.22,2.00,...,0.0,1.999702,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,801010,农林牧渔,2020-01-07,3531.88,3647.21,3530.23,3624.72,23.28,306.57,3.74,...,0.0,3.740974,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,801010,农林牧渔,2020-01-08,3634.48,3642.81,3559.93,3559.93,22.49,275.67,-1.79,...,0.0,-1.787448,4.859262,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,801010,农林牧渔,2020-01-09,3582.81,3606.12,3547.75,3566.77,22.31,237.45,0.19,...,0.0,0.192139,4.123766,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,801010,农林牧渔,2020-01-10,3565.13,3565.14,3487.57,3493.82,18.88,207.00,-2.05,...,0.0,-2.045268,-0.005438,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,801010,农林牧渔,2020-01-13,3480.89,3517.73,3470.31,3517.73,16.26,180.90,0.68,...,0.0,0.684351,-2.951676,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,801010,农林牧渔,2020-01-14,3515.16,3546.22,3494.20,3520.44,19.85,213.92,0.08,...,0.0,0.077038,-1.109291,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,801010,农林牧渔,2020-01-15,3538.07,3538.08,3458.79,3466.91,14.60,169.30,-1.52,...,0.0,-1.520549,-2.799732,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [32]:
BBIndex=allindex[allindex['index_code'].notna()]
BBIndex.loc[BBIndex.close>=BBIndex.open,'Signal']=0
BBIndex.loc[BBIndex.close<BBIndex.open,'Signal']=1
BBIndex['ChangeSignal']=BBIndex.Signal.diff().abs()
BBIndex.ChangeSignal.fillna(0,inplace=True) #解决首行0
BBIndex['BBperiod']=BBIndex.groupby('index_code').ChangeSignal.cumsum()

#板块级别日线涨跌周期

high = BBIndex.groupby(['index_name','BBperiod'])['high'].max().rename('high').reset_index()
low = BBIndex.groupby(['index_name','BBperiod'])['low'].min().rename('low').reset_index()
amount = BBIndex.groupby(['index_name','BBperiod'])['amount'].sum().rename('amount').reset_index()
open1 = BBIndex.groupby(['index_name','BBperiod'])['open'].first().rename('open').reset_index()
close1 = BBIndex.groupby(['index_name','BBperiod'])['close'].last().rename('close').reset_index()
lastperiod=BBIndex.groupby('index_name').BBperiod.max().rename('lastbar')

BBI2 = pd.merge(open1,close1,how='left',on=('index_name','BBperiod')) 
BBI2 = pd.merge(BBI2,high,how='left',on=('index_name','BBperiod')) 
BBI2 = pd.merge(BBI2,low,how='left',on=('index_name','BBperiod')) 
BBI2 = pd.merge(BBI2,amount,how='left',on=('index_name','BBperiod')) 
BBI2 = pd.merge(BBI2,lastperiod,how='left',on='index_name') 

close=BBI2.close.values
#swsf.date=swsf.date.values

diff, dea, my_macd = talib.MACD(close,
                                             fastperiod=12,
                                             slowperiod=26,
#获取均线的数据，通过timeperiod参数来分别获取 5,10,20 日均线的数据。
                                             signalperiod=9)
BBI2['D-diff']=diff
BBI2['D-dea']=dea
BBI2['D-MACD']=my_macd
BBI2=BBI2[BBI2['BBperiod']==BBI2['lastbar']]
BBI2 = BBI2.sort_values(by=['D-MACD'],axis=0,ascending=[False])

#BBI2.to_csv('C:/Users/atianth/Desktop/BBtest2.csv',index=False,encoding='utf-8-sig') 
BBI2

,index_name,BBperiod,open,close,high,low,amount,lastbar,D-diff,D-dea,D-MACD
901,家用电器,106.0,8779.82,9161.60,9229.11,8776.45,301.09,106.0,163.578050,118.779095,44.798955
1478,汽车,98.0,5073.66,5348.05,5423.32,5024.64,1383.61,98.0,196.585236,179.194781,17.390455
2510,银行,104.0,3483.94,3518.72,3545.01,3483.94,156.57,104.0,34.451241,24.857267,9.593974
1779,纺织服装,94.0,1938.61,1948.68,1960.33,1932.81,87.81,94.0,14.536711,12.530547,2.006163
2405,钢铁,100.0,2040.03,2063.78,2072.03,2037.34,164.49,100.0,-1.493073,-1.614425,0.121353
1886,综合,106.0,2390.00,2432.79,2443.40,2389.09,76.50,106.0,-13.575063,-12.576150,-0.998913
1379,机械设备,94.0,1415.10,1446.67,1451.95,1401.09,1707.90,94.0,15.100485,17.800866,-2.700381
1684,电气设备,106.0,6667.62,6988.18,7028.31,6587.48,2213.51,106.0,247.707066,251.038562,-3.331495
604,医药生物,100.0,11424.38,12076.86,12123.40,11304.22,2811.81,100.0,-4.355838,-0.548476,-3.807362
205,传媒,104.0,808.26,810.11,817.38,799.43,470.12,104.0,-7.484786,-3.510287,-3.974499


In [33]:
BBWIndex=allindex[allindex['index_code'].notna()]
BBWIndex['year']=BBWIndex.date.dt.isocalendar().year
BBWIndex['week']=BBWIndex.date.dt.isocalendar().week
high = BBWIndex.groupby(['index_name','year','week'])['high'].max().rename('high').reset_index()
low = BBWIndex.groupby(['index_name','year','week'])['low'].min().rename('low').reset_index()
amount = BBWIndex.groupby(['index_name','year','week'])['amount'].sum().rename('amount').reset_index()
open1 = BBWIndex.groupby(['index_name','year','week'])['open'].first().rename('open').reset_index()
close1 = BBWIndex.groupby(['index_name','year','week'])['close'].last().rename('close').reset_index()

BBWI2 = pd.merge(open1,close1,how='left',on=('index_name','year','week')) 
BBWI2 = pd.merge(BBWI2,high,how='left',on=('index_name','year','week')) 
BBWI2 = pd.merge(BBWI2,low,how='left',on=('index_name','year','week')) 
BBWI2 = pd.merge(BBWI2,amount,how='left',on=('index_name','year','week')) 

BBWI2.loc[BBWI2.close>=BBWI2.open,'Signal']=0
BBWI2.loc[BBWI2.close<BBWI2.open,'Signal']=1
BBWI2['ChangeSignal']=BBWI2.Signal.diff().abs()
BBWI2.ChangeSignal.fillna(0,inplace=True) #解决首行0
BBWI2['BBperiod']=BBWI2.groupby('index_name').ChangeSignal.cumsum()

high2 = BBWI2.groupby(['index_name','BBperiod'])['high'].max().rename('high').reset_index()
low2 = BBWI2.groupby(['index_name','BBperiod'])['low'].min().rename('low').reset_index()
amount2 = BBWI2.groupby(['index_name','BBperiod'])['amount'].sum().rename('amount').reset_index()
open2 = BBWI2.groupby(['index_name','BBperiod'])['open'].first().rename('open').reset_index()
close2 = BBWI2.groupby(['index_name','BBperiod'])['close'].last().rename('close').reset_index()
lastperiod=BBWI2.groupby('index_name').BBperiod.max().rename('lastbar')

BBWI3 = pd.merge(open2,close2,how='left',on=('index_name','BBperiod')) 
BBWI3 = pd.merge(BBWI3,high2,how='left',on=('index_name','BBperiod')) 
BBWI3 = pd.merge(BBWI3,low2,how='left',on=('index_name','BBperiod')) 
BBWI3 = pd.merge(BBWI3,amount2,how='left',on=('index_name','BBperiod')) 
BBWI3 = pd.merge(BBWI3,lastperiod,how='left',on='index_name') 

close=BBWI3.close.values
#swsf.date=swsf.date.values

diff, dea, my_macd = talib.MACD(close,
                                             fastperiod=12,
                                             slowperiod=26,
#获取均线的数据，通过timeperiod参数来分别获取 5,10,20 日均线的数据。
                                             signalperiod=9)
BBWI3['W-diff']=diff
BBWI3['W-dea']=dea
BBWI3['W-MACD']=my_macd
BBWI3=BBWI3[BBWI3['BBperiod']==BBWI3['lastbar']]
BBWI3 = BBWI3.sort_values(by=['W-MACD'],axis=0,ascending=[False])

#BBWI3.to_csv('C:/Users/atianth/Desktop/BBtest2.csv',index=False,encoding='utf-8-sig') 
BBWI3

,index_name,BBperiod,open,close,high,low,amount,lastbar,W-diff,W-dea,W-MACD
189,国防军工,22.0,1615.71,1511.57,1627.58,1501.32,1821.97,22.0,-520.468374,-719.150037,198.681664
260,建筑装饰,23.0,2107.61,2075.19,2124.10,2055.35,879.17,23.0,-737.773066,-931.373488,193.600422
639,食品饮料,19.0,23124.40,24556.99,24771.24,22951.11,1652.67,19.0,4684.239489,4492.808519,191.430971
660,餐饮旅游,20.0,9512.03,9715.49,9799.96,9109.28,181.43,20.0,-549.582858,-715.321954,165.739096
428,纺织服装,23.0,2053.47,1948.68,2072.88,1932.81,869.97,23.0,-492.116632,-618.708987,126.592355
404,电气设备,22.0,6667.62,6988.18,7028.31,6587.48,2213.51,22.0,620.193469,497.339492,122.853977
167,商业贸易,19.0,3715.56,3616.44,3818.65,3574.40,857.19,19.0,-969.326709,-1069.916457,100.589748
502,轻工制造,27.0,2592.77,2673.29,2678.13,2562.17,470.30,27.0,-187.651563,-280.901035,93.249472
335,机械设备,24.0,1415.10,1446.67,1451.95,1401.09,1707.90,24.0,-201.791367,-288.538284,86.746917
358,汽车,22.0,4746.38,5348.05,5423.32,4701.68,5701.20,22.0,617.277327,552.811662,64.465664


In [34]:
BBMIndex=allindex[allindex['index_code'].notna()]
BBMIndex['year']=BBMIndex.date.dt.isocalendar().year
BBMIndex['month']=BBMIndex.date.dt.month
high = BBMIndex.groupby(['index_name','year','month'])['high'].max().rename('high').reset_index()
low = BBMIndex.groupby(['index_name','year','month'])['low'].min().rename('low').reset_index()
amount = BBMIndex.groupby(['index_name','year','month'])['amount'].sum().rename('amount').reset_index()
open1 = BBMIndex.groupby(['index_name','year','month'])['open'].first().rename('open').reset_index()
close1 = BBMIndex.groupby(['index_name','year','month'])['close'].last().rename('close').reset_index()

BBMI2 = pd.merge(open1,close1,how='left',on=('index_name','year','month')) 
BBMI2 = pd.merge(BBMI2,high,how='left',on=('index_name','year','month')) 
BBMI2 = pd.merge(BBMI2,low,how='left',on=('index_name','year','month')) 
BBMI2 = pd.merge(BBMI2,amount,how='left',on=('index_name','year','month')) 

BBMI2.loc[BBMI2.close>=BBMI2.open,'Signal']=0
BBMI2.loc[BBMI2.close<BBMI2.open,'Signal']=1
BBMI2['ChangeSignal']=BBMI2.Signal.diff().abs()
BBMI2.ChangeSignal.fillna(0,inplace=True) #解决首行0
BBMI2['BBperiod']=BBMI2.groupby('index_name').ChangeSignal.cumsum()

high2 = BBMI2.groupby(['index_name','BBperiod'])['high'].max().rename('high').reset_index()
low2 = BBMI2.groupby(['index_name','BBperiod'])['low'].min().rename('low').reset_index()
amount2 = BBMI2.groupby(['index_name','BBperiod'])['amount'].sum().rename('amount').reset_index()
open2 = BBMI2.groupby(['index_name','BBperiod'])['open'].first().rename('open').reset_index()
close2 = BBMI2.groupby(['index_name','BBperiod'])['close'].last().rename('close').reset_index()
lastperiod=BBMI2.groupby('index_name').BBperiod.max().rename('lastbar')

BBMI3 = pd.merge(open2,close2,how='left',on=('index_name','BBperiod')) 
BBMI3 = pd.merge(BBMI3,high2,how='left',on=('index_name','BBperiod')) 
BBMI3 = pd.merge(BBMI3,low2,how='left',on=('index_name','BBperiod')) 
BBMI3 = pd.merge(BBMI3,amount2,how='left',on=('index_name','BBperiod')) 
BBMI3 = pd.merge(BBMI3,lastperiod,how='left',on='index_name') 

close=BBMI3.close.values
#swsf.date=swsf.date.values

diff, dea, my_macd = talib.MACD(close,
                                             fastperiod=12,
                                             slowperiod=26,
#获取均线的数据，通过timeperiod参数来分别获取 5,10,20 日均线的数据。
                                             signalperiod=9)
BBMI3['M-diff']=diff
BBMI3['M-dea']=dea
BBMI3['M-MACD']=my_macd
BBMI3=BBMI3[BBMI3['BBperiod']==BBMI3['lastbar']]
BBMI3 = BBMI3.sort_values(by=['M-MACD'],axis=0,ascending=[False])

#BBMI3.to_csv('C:/Users/atianth/Desktop/BBtest2.csv',index=False,encoding='utf-8-sig') 
BBMI3

,index_name,BBperiod,open,close,high,low,amount,lastbar,M-diff,M-dea,M-MACD
150,食品饮料,3.0,23244.01,24556.99,24771.24,22951.11,5059.58,3.0,4408.394728,1807.610469,2600.784259
42,家用电器,5.0,8505.35,9161.60,9229.11,8505.35,1797.11,5.0,1246.750848,880.541574,366.209273
106,计算机,5.0,6093.38,5464.08,6300.91,5395.60,33988.94,5.0,542.494981,179.138484,363.356497
79,汽车,2.0,3671.94,5348.05,5423.32,3470.45,37036.48,2.0,-144.229777,-477.428490,333.198713
89,电气设备,2.0,4298.54,6988.18,7317.67,4266.70,58815.90,2.0,534.570141,204.354263,330.215877
86,电子元器件,6.0,4551.99,4586.64,4775.09,4421.53,11857.89,6.0,271.555160,10.595987,260.959173
139,银行,8.0,3367.16,3518.72,3678.81,3349.45,2865.54,8.0,208.398906,48.782518,159.616388
64,房地产,7.0,4255.77,3943.84,4355.98,3893.69,5298.75,7.0,-314.778204,-407.991970,93.213766
49,建筑材料,7.0,8530.98,7783.64,8530.98,7478.99,3905.23,7.0,999.585887,961.126202,38.459685
70,有色金属,5.0,3196.68,3282.37,3334.55,3190.34,3048.99,5.0,-353.815368,-383.939487,30.124119


In [ ]:
IndexReport=swsf['index_name','close',]

In [ ]:

#从TUSHARE获取申万一级行业列表及其对应股票
Sdf1 = pro.index_classify(level='L1', src='SW')
Sdf1.to_csv('C:/quant/swsindex/L1.csv',index=False,encoding='utf-8-sig') 
#获取申万二级行业列表
Sdf2 = pro.index_classify(level='L2', src='SW')
Sdf2.to_csv('C:/quant/swsindex/L2.csv',index=False,encoding='utf-8-sig')

#获取申万三级级行业列表
Sdf3 = pro.index_classify(level='L3', src='SW')
Sdf3.to_csv('C:/quant/swsindex/L3.csv',index=False,encoding='utf-8-sig') 

#Sdf3.head(10)
#SW1=pd.merge(Sdf1,Sdf2,how='right',on=['index_code','index_code'])
#SW2=pd.merge(SW1,Sdf2,how='right',on=['index_code','index_code'])


In [ ]:

#从TUSHARE端口获取成分股
ISindex = pd.DataFrame() 

for i in Sdf1.index_code:
    time.sleep(0.7)
    Tf=pro.index_member(index_code=i)    
    ISindex = ISindex.append(Tf)

ISindex.to_csv(r'C:/quant/swsindex/total.csv') 
ISindex.rename(columns={'con_code':'ts_code'}, inplace = True)
ISindex=pd.merge(ISindex,Sdf1,how='left',on='index_code')
ISindex=ISindex.drop(["in_date",'out_date','level'],axis=1)

ISindex
    


In [ ]:
latestday=max(swsf.date)
ETF=swsf[swsf.date==latestday]
ETF=ETF[['index_code','K1','Indextrendscore-L']]
ISindex['index_code']=ISindex['index_code'].str.split('.', 2).str[0]

ISindex2 = pd.merge(ISindex, ETF, how='left',on='index_code') #分段信息
ISindex2

In [ ]:

#从tushare下载日数据

for i in range(20200901,20200932):
    time.sleep(0.7)
    print (i) 
    x=str(i)
    df = pro.daily(trade_date=x) #个股数据
    Adf = pro.query('adj_factor', trade_date=x) #复权因子
    Adf.rename(columns={'trade_date':'trade_date_x'}, inplace = True)
    Adf.adj_factor.fillna(0,inplace=True)


    Mdf = pro.moneyflow(trade_date=x) #个股资金流向，出韭菜指数
    X1=pd.merge(df,Mdf,on='ts_code')
    X1=pd.merge(X1,Adf,on=['ts_code','trade_date_x'])

    mf = pro.margin(trade_date=x)#融资融券数据
    GCdf = pro.repo_daily(ts_code='204001.sh',trade_date=x) # 逆回购数据
    #HSdf = pro.repo_daily(ts_code='000906.sh',trade_date=x) # 逆回购数据

    #X2=pd.merge(mf,GCdf,on='trade_date')

    #df2 = pro.index_dailybasic(start_date=x,fields='ts_code,trade_date,turnover_rate,pe,pe_ttm')
    #pe = df2['pe_ttm']


    writer1 = X1.to_csv('C:/quant/daydate/2020/'+x + '.csv') 
    writer2 = GCdf.to_csv('C:/quant/GCdata/2020/'+x + '.csv') 
    writer3 = mf.to_csv('C:/quant/financing/2020/'+x + '.csv') #要晚一天出，所以不能合并
    #writer4 = HSdf.to_csv('C:/quant/index/2020/'+x + '.csv') #要晚一天出，所以不能合并


   # df.to_csv(writer)
  #  writer.save()
   # writer.close()
    


In [ ]:
import pandas as pd
import datetime
import numpy as np
from os import walk
for root,dirs,files in walk(r'C:/quant/daydate/2020/',topdown=False):
    num = len(files)
    alldata = pd.DataFrame() 
for i in range(num):
    newdata = pd.read_csv(r'C:/quant/daydate/2020/%s'%files[i])
    
    alldata = alldata.append(newdata)
alldata = alldata.sort_values(by=['ts_code','trade_date_x'],axis=0,ascending=[True,True])
alldata.reset_index(drop=True, inplace=True)


In [ ]:
latestday=alldata.trade_date_x.max()
alldata.adj_factor.fillna(1,inplace=True)
alldata.loc[alldata['trade_date_x']==latestday,'now_adj']=alldata['adj_factor'] # 最新复权因子
alldata.now_adj=alldata.now_adj.fillna(method='bfill')

#alldata.loc[alldata['ts_code']==alldata['ts_code'].shift(1),'now_adj']=alldata.now_adj.shift(1) # 最新复权因子

alldata['close_qfq'] = alldata['close'] * alldata['adj_factor'] / alldata.now_adj  # 前复权 当日收盘价 × 当日复权因子 / 最新复权因子
alldata.to_csv('C:/quant/index/2020/test.csv') 
alldata.shape
alldata

In [ ]:
from scipy.stats import rankdata
df=alldata[alldata['ts_code'].notna()]

close=df.close_qfq
diff, dea, my_macd = talib.MACD(close,
                                             fastperiod=12,
                                             slowperiod=26,
#获取均线的数据，通过timeperiod参数来分别获取 5,10,20 日均线的数据。
                                             signalperiod=9)
SMA5=talib.SMA(close,timeperiod=5)
SMA10=talib.SMA(close,timeperiod=10)
SMA20=talib.SMA(close,timeperiod=20)
SMA60=talib.SMA(close,timeperiod=60)
SMA120=talib.SMA(close,timeperiod=120)

EMA5=talib.EMA(close,timeperiod=5)
EMA10=talib.EMA(close,timeperiod=10)
EMA20=talib.EMA(close,timeperiod=20)
EMA60=talib.EMA(close,timeperiod=60)
EMA120=talib.EMA(close,timeperiod=120)

df['diff']=diff
df['dea']=dea
df['MACD']=my_macd
df['SMA5']=SMA5
df['SMA10']=SMA10
df['SMA20']=SMA20
df['SMA60']=SMA60
df['SMA120']=SMA120

df['EMA5']=EMA5
df['EMA10']=EMA10
df['EMA60']=EMA60
df['EMA120']=EMA120

df['EMA20']=EMA20
df['SMACD-K']=(diff-dea)/dea
df['C/S']=(close/SMA20-1)*100
df['M/L']=(SMA60/SMA120-1)*100
df['S/M']=(SMA20/SMA60-1)*100
df.reset_index(drop=True, inplace=True)

df.loc[df['ts_code'] == df['ts_code'].shift(4),'Chg5D']= (df.close_qfq/df.close_qfq.shift(4)-1)*100
df.loc[df['ts_code'] == df['ts_code'].shift(9),'Chg10D']= (df.close_qfq/df.close_qfq.shift(9)-1)*100
df.loc[df['ts_code'] == df['ts_code'].shift(19),'Chg20D']= (df.close_qfq/df.close_qfq.shift(19)-1)*100
df.loc[df['ts_code'] == df['ts_code'].shift(59),'Chg60D']= (df.close_qfq/df.close_qfq.shift(59)-1)*100
df.loc[df['ts_code'] == df['ts_code'].shift(119),'Chg120D']= (df.close_qfq/df.close_qfq.shift(119)-1)*100





df['ATR'] = talib.ATR(df.high.values,df.low.values,df.pre_close.values, timeperiod=20)


df.fillna(0,inplace=True) #去除null
df.loc[(df['close_qfq']>df['EMA20']) & (df['close_qfq']>df['SMA20']) & (df['SMA20']>df['SMA20'].shift(19)),'kl1']=1 #破线拐头
df.loc[(df['EMA20']>df['EMA60']) & (df['close_qfq']>df['EMA20'].shift(19)),'K2']=1 #双均线交叉
df.loc[(df['EMA20']>df['SMA60']) & (df['close_qfq']>df['EMA20'].shift(19)),'K3']=1 
df.loc[(df['SMA20']>df['EMA60']) & (df['close_qfq']>df['SMA20'].shift(19)),'K4']=1 
df.loc[(df['SMA20']>df['SMA60']) & (df['close_qfq']>df['SMA20'].shift(19)),'K5']=1 

df.loc[(df['close_qfq']<df['EMA20']) & (df['close_qfq']<df['SMA20']) & (df['SMA20']<df['SMA20'].shift(19)),'kl1']=-1 #空头方向破线拐头
df.loc[(df['EMA20']<df['EMA60']) & (df['close_qfq']<df['EMA20'].shift(19)),'K2']=-1 #双均线交叉
df.loc[(df['EMA20']<df['SMA60']) & (df['close_qfq']<df['EMA20'].shift(19)),'K3']=-1 
df.loc[(df['SMA20']<df['EMA60']) & (df['close_qfq']<df['SMA20'].shift(19)),'K4']=-1 
df.loc[(df['SMA20']<df['SMA60']) & (df['close_qfq']<df['SMA20'].shift(19)),'K5']=-1 
df.fillna(0,inplace=True) #去除null
df['trendscore-L']=(df.kl1+df.K2+df.K3+df.K4+df.K5)

df.loc[(df['close_qfq']>df['EMA10']) & (df['close_qfq']>df['SMA10']) & (df['SMA10']>df['SMA10'].shift(19)),'ks1']=1 #破线拐头
df.loc[(df['EMA10']>df['EMA20']) & (df['close_qfq']>df['EMA10'].shift(19)),'KK2']=1 #双均线交叉
df.loc[(df['EMA10']>df['SMA20']) & (df['close_qfq']>df['EMA10'].shift(19)),'KK3']=1 
df.loc[(df['SMA10']>df['EMA20']) & (df['close_qfq']>df['SMA10'].shift(19)),'KK4']=1 
df.loc[(df['SMA10']>df['SMA20']) & (df['close_qfq']>df['SMA10'].shift(19)),'KK5']=1 

df.loc[(df['close_qfq']<df['EMA10']) & (df['close_qfq']<df['SMA10']) & (df['SMA10']<df['SMA10'].shift(19)),'ks1']=-1 #空头方向破线拐头
df.loc[(df['EMA10']<df['EMA20']) & (df['close_qfq']<df['EMA10'].shift(19)),'KK2']=-1 #双均线交叉
df.loc[(df['EMA10']<df['SMA20']) & (df['close_qfq']<df['EMA10'].shift(19)),'KK3']=-1 
df.loc[(df['SMA10']<df['EMA20']) & (df['close_qfq']<df['SMA10'].shift(19)),'KK4']=-1 
df.loc[(df['SMA10']<df['SMA20']) & (df['close_qfq']<df['SMA10'].shift(19)),'KK5']=-1 
df.fillna(0,inplace=True) #去除null
df['trendscore-S']=(df.ks1+df.KK2+df.KK3+df.KK4+df.KK5)
df = pd.merge(df, ISindex, how='left',on='ts_code') 

df=df[['trade_date_x','ts_code','close_qfq','amount','pct_chg','ATR','Chg5D','Chg10D','Chg20D','Chg60D','Chg120D','C/S','S/M','M/L','ks1','kl1','trendscore-S','trendscore-L']]

df.head(10)


In [ ]:
latestday=max(df.trade_date_x)
valve=150000   #以1亿5000万元为界限，150000K
df2=df[(df.trade_date_x==latestday) & (df.amount > valve)]
df2=pd.merge(df2,pool,how='left',on='ts_code')
df2=df2.drop(df2.columns[-2:-1],axis=1)
df2=pd.merge(df2,ISindex2,how='left',on='ts_code')
df2=df2.drop(["symbol",'area','list_date','index_code'],axis=1)
df2.rename(columns={'industry_name':'industry_L1','industry':'industry_L2'}, inplace = True)
order = ['trade_date_x', 'ts_code', 'name', 'industry_L1', 'industry_L2', 'K1', 'Indextrendscore-L', 'close_qfq','amount','pct_chg','ATR','Chg5D','Chg10D','Chg20D','Chg60D','Chg120D','C/S','S/M','M/L','ks1','kl1','trendscore-S','trendscore-L']
df2 = df2[order]
df2


In [ ]:
#找长期，中期乖离率高，但短期下杀严重，趋势却走好的
DF=df2[(df2['C/S']<0) & (df2['S/M']>5) & (df2['M/L']> 5) & (df2['trendscore-S']>=3) & (df2['ks1']==1)]
DF

In [ ]:
#writer = pd.ExcelWriter(r'C:/quant/output/combinedoutput.csv')
df2.to_csv(r'C:/quant/output/dailyoutput.csv',index=False,encoding='utf-8-sig')
#这里“AllData”是sheet的名字
#writer.save()#执行完这一步之后，合并后的表格就保存在了C:\Users\output.xls中

In [ ]:
#美债利率

#获取5年期和20年期数据
Udf = pro.us_tltr(start_date='20180101', end_date='20200907', fields='ltc,cmt')
Udf.to_csv("C:/quant/index/UDF.csv", index=False)


In [ ]:
#可转债
EEf = pro.cb_daily(trade_date='20200821')

EFF= pro.cb_basic(fields="ts_code,bond_short_name,stk_code,stk_short_name,list_date,delist_date")   

    


EFF.to_csv(r'C:/quant/debt/2020/20200821.csv',index=False,encoding='utf-8-sig')
 

    


In [ ]:

#取000001的前复权行情
EEf = ts.pro_bar(ts_code='000001.SZ', adj='qfq', start_date='20200101', end_date='20201029')
EEf.to_csv(r'C:/quant/output/testoutput.csv',index=False,encoding='utf-8-sig')
